In [1]:
#!pip install keras
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Add, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import glorot_uniform
from keras.layers.merge import concatenate
from keras import optimizers
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
num_classes  = 10
batch_size   = 128
epochs       = 50
dropout      = 0.5
weight_decay = 0.0001

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255

In [4]:
def inception_module(X, f1, f2_in, f2_out, f3_in, f3_out, f4_out, stage, block):
    
    name_sb = str(stage) + str(block)
    
    # 1x1 conv
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu', name = 'conv_tower1_' + name_sb)(X)
    # 3x3 conv
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu', name = 'conv_tower2_1_' + name_sb)(X)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu', name = 'conv_tower2_2' + name_sb)(conv3)
    # 5x5 conv
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu', name = 'conv_tower3_1_' + name_sb)(X)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu', name = 'conv_tower3_2_' + name_sb)(conv5)
    # 3x3 max pooling
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same', name = 'mpool_tower4_' + name_sb)(X)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu', name = 'conv_tower4_' + name_sb)(pool)
    # concatenate filters, assumes filters/channels last
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out

In [5]:
def inception_v1(input_shape, classes):
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(32, (3, 3), strides = (2, 2), name = 'conv_pre_inception_1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_1')(X)
    
    # Stage 2
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_2', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_2')(X)
    X = Activation('relu')(X)
    X = Conv2D(192, (3, 3), strides = (1, 1), name = 'conv_pre_inception_3', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_pre_inception_3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_pre_inception_2')(X)
    
    # Stage 3
    X = inception_module(X, 64, 96, 128, 16, 32, 32, 3, 'a')
    X = inception_module(X, 128, 128, 192, 32, 96, 64, 3, 'b')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_3')(X)
    
    # Stage 4
    X = inception_module(X, 192, 96, 208, 16, 48, 64, 4, 'a')
    X = inception_module(X, 160, 112, 224, 24, 64, 64, 4, 'b')
    X = inception_module(X, 128, 128, 256, 24, 24, 64, 4, 'c')
    X = inception_module(X, 112, 144, 288, 32, 64, 64, 4, 'd')
    X = inception_module(X, 256, 120, 320, 32, 128, 128, 4, 'e')
    X = MaxPooling2D((2, 2), strides=(1, 1), name = 'mpool_inception_4')(X)
    
    # Stage 5
    X = inception_module(X, 256, 160, 320, 32, 128, 128, 5, 'a')
    X = inception_module(X, 384, 192, 384, 48, 128, 128, 5, 'b')
    X = AveragePooling2D((7, 7), strides=(1, 1), name = 'apool_inception_5')(X)
    
    # Stage 6
    X = Dropout(0.4)(X)
    X = Flatten()(X)
    X = Dense(1000, activation='relu', kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    inceptionv1 = Model(inputs = X_input, outputs = X, name='InceptionV1')

    return inceptionv1

In [6]:
inception = inception_v1(x_train.shape[1:], num_classes)
inception.summary()

Model: "InceptionV1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_pre_inception_1 (Conv2D)   (None, 18, 18, 32)   896         zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_pre_inception_1 (BatchNormal (None, 18, 18, 32)   128         conv_pre_inception_1[0][0]       
________________________________________________________________________________________

In [7]:
inception.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
inception.fit(x_train, y_train, epochs = 20, batch_size = 128)

Epoch 1/20
50000/50000 [==============================] - 89s 2ms/step - loss: 1.7325 - accuracy: 0.3360
Epoch 2/20
50000/50000 [==============================] - 79s 2ms/step - loss: 1.2253 - accuracy: 0.5521
Epoch 3/20
50000/50000 [==============================] - 78s 2ms/step - loss: 0.9917 - accuracy: 0.6452
Epoch 4/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.8390 - accuracy: 0.7027
Epoch 5/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.7287 - accuracy: 0.7463 0s - loss: 0.7286 - accuracy: 0.74
Epoch 6/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.6365 - accuracy: 0.7807
Epoch 7/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.5679 - accuracy: 0.8060
Epoch 8/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.4980 - accuracy: 0.8301
Epoch 9/20
50000/50000 [==============================] - 77s 2ms/step - loss: 0.4565 - accuracy: 0.8443
Epoch 10/20
50000/50

In [9]:
scores = inception.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 7s 743us/step
Test loss: 0.9519953559875488
Test accuracy: 0.7540000081062317


In [10]:
inception.save('inceptionv1.h5')
inception.save_weights('inceptionv1_weights.h5')